In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import logging

logging.basicConfig(
    format="%(asctime)s ; %(levelname)s ; %(message)s",
    level=logging.DEBUG
)
logging.getLogger("scapy").setLevel(logging.CRITICAL)
logger = logging.getLogger("adAPT")

from typing import Tuple
from pathlib import Path
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.regularizers import L2
from tensorflow.keras import losses
from tensorflow.keras import metrics
from tensorflow.keras import optimizers
from tensorflow import keras
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from collections import Counter


2023-05-12 22:51:08.199775: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-12 22:51:17,011 ; DEBUG ; Creating converter from 7 to 5
2023-05-12 22:51:17,012 ; DEBUG ; Creating converter from 5 to 7
2023-05-12 22:51:17,013 ; DEBUG ; Creating converter from 7 to 5
2023-05-12 22:51:17,014 ; DEBUG ; Creating converter from 5 to 7


In [3]:
b_pkl = "./data/benign_features.pkl"
m_pkl = "./data/malicious_features.pkl"

b_df = pd.read_pickle(b_pkl)
m_df = pd.read_pickle(m_pkl)


In [4]:
all_df = pd.concat([b_df, m_df])

In [5]:
all_df.sample(5)

,protocol,app_layer,source_addr,dest_addr,source_port,dest_port,proto_packet_length,proto_packet_cache,ip_packet_length,ip_packet_cache,...,base_domain_entropy,host_length,proto_packet_entropy,source_ip_class_a,source_ip_class_b,source_ip_class_c,dest_ip_class_a,dest_ip_class_b,dest_ip_class_c,malware
8990,IP,Unknown,51.195.169.87,10.1.11.101,8080,64823,40,"b'E\x00\x00(%[\x00\x00\x80\x06""\xf53\xc3\xa9W\...",40,"b'E\x00\x00(%[\x00\x00\x80\x06""\xf53\xc3\xa9W\...",...,0.0,0,3.921928,51,51.195,51.195.169,10,10.1,10.1.11,1.0
48305,IP,Unknown,162.215.96.249,172.16.1.137,465,64176,40,b'E\x00\x00(\n\xfa\x00\x00\x80\x06~l\xa2\xd7`\...,40,b'E\x00\x00(\n\xfa\x00\x00\x80\x06~l\xa2\xd7`\...,...,0.0,0,3.921928,162,162.215,162.215.96,172,172.16,172.16.1,1.0
3596,IP,Unknown,10.12.7.101,139.177.146.137,50113,443,193,b'E\x00\x00\xc1\x84\xe5@\x00\x80\x06E\xa6\n\x0...,193,b'E\x00\x00\xc1\x84\xe5@\x00\x80\x06E\xa6\n\x0...,...,0.0,0,3.984184,10,10.12,10.12.7,139,139.177,139.177.146,1.0
26313,IP,Unknown,182.162.143.56,172.16.1.137,443,64110,1416,b'E\x00\x05\x88\xdb\x18\x00\x00\x80\x06f\xe3\x...,1416,b'E\x00\x05\x88\xdb\x18\x00\x00\x80\x06f\xe3\x...,...,0.0,0,4.084184,182,182.162,182.162.143,172,172.16,172.16.1,1.0
14606,IP,HTTPResponse,178.62.112.199,10.3.18.101,8080,51292,1500,b'E\x00\x05\xdc\xa3u\x00\x00\x80\x06R9\xb2>p\x...,1500,b'E\x00\x05\xdc\xa3u\x00\x00\x80\x06R9\xb2>p\x...,...,0.0,0,4.084184,178,178.62,178.62.112,10,10.3,10.3.18,1.0


In [7]:
def prepare_df_for_ml(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    fields = [
        "protocol",
        "app_layer",
        "source_addr", 
        "dest_addr",
        "source_port",
        "dest_port",
        "proto_packet_length",
        "ip_packet_length",
        # "url",
        "url_entropy",
        "host_entropy",
        "base_domain_entropy",
        "host_length",
        "proto_packet_entropy",
    ]
    new_df = df[fields]
    new_df.loc[:, ["source_port", "dest_port"]] = new_df[["source_port", "dest_port", ]].astype(str)
    new_df.loc[:, ["ip_packet_length", "source_port"]] = new_df[["ip_packet_length", "source_port", ]].astype(float)
        
    return pd.get_dummies(new_df)

In [8]:
y = all_df["malware"].values
input_values = all_df.drop("malware", axis=1)
prepped = prepare_df_for_ml(input_values)
X = np.asarray(prepped.values).astype("float64")

In [ ]:
y.shape

(196981,)

In [ ]:
X.shape

(196981, 6057)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.40, random_state=234)

In [ ]:
X_test, X_cv, Y_test, Y_cv = train_test_split(X_test, Y_test, test_size=0.50, random_state=33)

In [ ]:
print(f"X_train.shape: {X_train.shape}")
print(f"X_test.shape: {X_test.shape}")
print(f"X_cv.shape: {X_cv.shape}")
print(f"Y_train.shape: {Y_train.shape}")
print(f"Y_test.shape: {Y_test.shape}")
print(f"Y_cv.shape: {Y_cv.shape}")


X_train.shape: (118188, 6057)
X_test.shape: (39396, 6057)
X_cv.shape: (39397, 6057)
Y_train.shape: (118188,)
Y_test.shape: (39396,)
Y_cv.shape: (39397,)


In [ ]:
normalizer = tf.keras.layers.Normalization(axis=-1)

layer_1 = layers.Dense(units=9, input_shape=(X_train.shape[-1], ), activation="relu", kernel_regularizer=L2(0.01))
layer_2 = layers.Dense(units=15, activation="relu", kernel_regularizer=L2(0.01))
layer_3 = layers.Dense(units=1, activation="sigmoid", kernel_regularizer=L2(0.01))

model = keras.Sequential([
    normalizer,
    layer_1,
    layer_2,
    layer_3
])

In [ ]:
model.compile(
    optimizer="adam",  # Optimizer
    # Loss function to minimize
    loss=losses.BinaryCrossentropy(),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(),
        tf.keras.metrics.FalseNegatives()
    ],
)

In [ ]:
model.fit(X_train, Y_train, epochs=10)

Epoch 1/10
3694/3694 [==============================] - 15s 4ms/step - loss: 0.3199 - binary_accuracy: 0.9193 - false_negatives: 604.0000
Epoch 2/10
3694/3694 [==============================] - 13s 4ms/step - loss: 0.2130 - binary_accuracy: 0.9684 - false_negatives: 1117.0000
Epoch 3/10
3694/3694 [==============================] - 13s 4ms/step - loss: 0.2084 - binary_accuracy: 0.9685 - false_negatives: 1141.0000
Epoch 4/10
3694/3694 [==============================] - 13s 4ms/step - loss: 0.2007 - binary_accuracy: 0.9694 - false_negatives: 1117.0000
Epoch 5/10
3694/3694 [==============================] - 13s 4ms/step - loss: 0.1938 - binary_accuracy: 0.9707 - false_negatives: 1062.0000
Epoch 6/10
3694/3694 [==============================] - 14s 4ms/step - loss: 0.1913 - binary_accuracy: 0.9709 - false_negatives: 1026.0000
Epoch 7/10
3694/3694 [==============================] - 13s 4ms/step - loss: 0.1995 - binary_accuracy: 0.9699 - false_negatives: 1128.0000
Epoch 8/10
3694/3694 [======

In [ ]:
print("Evaluate on test data")
results = model.evaluate(X_test, Y_test)
print("test loss, test acc:", results)


Evaluate on test data
1232/1232 [==============================] - 3s 2ms/step - loss: 0.1935 - binary_accuracy: 0.9674 - false_negatives: 170.0000
test loss, test acc: [0.19354166090488434, 0.9673570990562439, 170.0]


In [ ]:
print(f"cv loss and acc: {model.evaluate(X_cv, Y_cv)}")

1232/1232 [==============================] - 3s 2ms/step - loss: 0.1944 - binary_accuracy: 0.9676 - false_negatives: 164.0000
cv loss and acc: [0.19438008964061737, 0.9676371216773987, 164.0]
